In [1]:
import oandapyV20
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.instruments as instruments
import json
import pandas as pd
import datetime as dt

In [2]:
cols = ['Pair', 'MOMENTUM_SCORE']
momentum_scoreboard = pd.DataFrame(columns=cols)


In [3]:
client = oandapyV20.API(access_token="0ca9619de6d428b6fdbcb4d20ef81268-cca1610e83cde7da5395d4468c2c1fe9")

pairs = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'USD_CAD', 'AUD_USD', 'NZD_USD']

for pair in pairs:
    cols = ['Date', 'Close']
    monthlyPrices = pd.DataFrame(columns=cols)
    print(pair)
    r = instruments.InstrumentsCandles(instrument=pair,
                                   params={"count": 3650, "granularity": "D"})
    raw_data = client.request(r)
    candles = raw_data['candles']
    #print(monthly_candles)
    for m in candles:
        new_row = {'Date': dt.datetime.strptime(m['time'][:10], '%Y-%m-%d'), 
                   'Close': float(m['mid']['c'])}
        monthlyPrices=monthlyPrices.append(new_row, ignore_index=True)
    
    #print(monthlyPrices)
    
    monthlyPrices['BEFORE_1M'] = monthlyPrices.shift(1)['Close']
    monthlyPrices['BEFORE_6M'] = monthlyPrices.shift(6)['Close']
    monthlyPrices['BEFORE_12M'] = monthlyPrices.shift(12)['Close']
    monthlyPrices.dropna(inplace=True)
    
    #print(monthlyPrices.head())
    
    for x in monthlyPrices.index:
        monthlyPrices.loc[x,'MOMENTUM_1M'] = monthlyPrices.loc[x,'Close'] - monthlyPrices.loc[x,'BEFORE_1M']
        monthlyPrices.loc[x,'MOMENTUM_6M'] = monthlyPrices.loc[x,'Close'] - monthlyPrices.loc[x,'BEFORE_6M']
        monthlyPrices.loc[x,'MOMENTUM_12M'] = monthlyPrices.loc[x,'Close'] - monthlyPrices.loc[x,'BEFORE_12M']
        
    for x in monthlyPrices.index:
        monthlyPrices.loc[x, 'MOMENTUM_INDEX'] = monthlyPrices.loc[x, 'MOMENTUM_1M'] * 0.5 + monthlyPrices.loc[x, 'MOMENTUM_6M'] * 0.3 + monthlyPrices.loc[x, 'MOMENTUM_12M'] * 0.2
    
    #print(monthlyPrices.iloc[-1]['MOMENTUM_INDEX'])
    momentum_scoreboard = momentum_scoreboard.append({'Pair': pair, 'MOMENTUM_SCORE': monthlyPrices.iloc[-1]['MOMENTUM_INDEX']}, ignore_index=True)


EUR_USD
GBP_USD
USD_JPY
USD_CAD
AUD_USD
NZD_USD


In [4]:
momentum_scoreboard

,Pair,MOMENTUM_SCORE
0,EUR_USD,0.064082
1,GBP_USD,0.063120
2,USD_JPY,-2.935500
3,USD_CAD,-0.044084
4,AUD_USD,0.054640
5,NZD_USD,0.039210


In [5]:
momentum_scoreboard.set_index('Pair', inplace=True)

In [6]:
momentum_scoreboard

,MOMENTUM_SCORE
Pair,
EUR_USD,0.064082
GBP_USD,0.063120
USD_JPY,-2.935500
USD_CAD,-0.044084
AUD_USD,0.054640
NZD_USD,0.039210


In [7]:
momentum_scoreboard.sort_values(by='MOMENTUM_SCORE')

,MOMENTUM_SCORE
Pair,
USD_JPY,-2.935500
USD_CAD,-0.044084
NZD_USD,0.039210
AUD_USD,0.054640
GBP_USD,0.063120
EUR_USD,0.064082
